# Training module

In [1]:
#import library
import csv
import pandas as pd
import pickle 
import numpy as np
import cv2
import mediapipe as mp
import time
import shutil
import os
import sys

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score 

# Test Camera

In [2]:
# Grabbing the Holistic Model from Mediapipe and
mp_holistic = mp.solutions.holistic

# Initializing the Model
holistic_model = mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
 
# Initializing the drawing utils for drawing the facial landmarks on image
mp_drawing = mp.solutions.drawing_utils

# Create datasheet

In [68]:
prj_path = r"C:\Users\Leopard\Desktop\myCode\Project ITM\Web-applications-for-elderly-physical-exercises\ElderFitmodel"

In [70]:
model_name = "lag_raises"

In [84]:
class_name = "lag_raises_3"

#create folder of the model
modelFolerPath = prj_path+"\Dataset/"+model_name
if not os.path.exists(modelFolerPath):
    os.makedirs(modelFolerPath)

In [85]:
# Absolute path of a file
go_src_path = prj_path+"\Dataset\predataset.csv"
go_dst_path = prj_path+"\Dataset\\"+model_name+"\predataset.csv"
data_path = prj_path+"\Dataset\\"+model_name+"\\"+class_name+".csv"

# Copying and rename the file
if not os.path.isfile(data_path):
    shutil.copy(go_src_path, go_dst_path)
    os.rename(go_dst_path, data_path)

In [86]:
# (0) in VideoCapture is used to connect to your computer's default camera
capture = cv2.VideoCapture(0)
 
# Initializing current time and precious time for calculating the FPS
previousTime = 0
currentTime = 0

while capture.isOpened():
    # capture frame by frame
    ret, frame = capture.read()
    
    # resizing the frame for better view
    frame = cv2.resize(frame, (640,480))
 
    # Converting the from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
 
    # Making predictions using holistic model
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = holistic_model.process(image)
    image.flags.writeable = True
 
    # Converting back the RGB image to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
 
    # Drawing Right hand Land Marks
    # mp_drawing.draw_landmarks(
    #     image,
    #     results.right_hand_landmarks,
    #     mp_holistic.HAND_CONNECTIONS
    # )
    
    # # Drawing Left hand Land Marks
    # mp_drawing.draw_landmarks(
    #     image,
    #     results.left_hand_landmarks,
    #     mp_holistic.HAND_CONNECTIONS
    # )
    
    mp_drawing.draw_landmarks(
        image, 
        results.pose_landmarks, 
        mp_holistic.POSE_CONNECTIONS,            
    )
    # Calculating the FPS
    currentTime = time.time()
    fps = 1 / (currentTime-previousTime)
    previousTime = currentTime
    
    
    # Displaying FPS on the image
    cv2.putText(image, str(int(fps))+" FPS", (10, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
 
    # Display the resulting image
    cv2.imshow("Test Camera", image)

    # Enter key 'q' to break the loop
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break
 
# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()

# Data acquisition

In [87]:
# (0) in VideoCapture is used to connect to your computer's default camera
capture = cv2.VideoCapture(0)
 
while capture.isOpened():
    # capture frame by frame
    ret, frame = capture.read()
 
    # resizing the frame for better view
    frame = cv2.resize(frame, (880,720))
 
    # Converting the from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
 
    # Making predictions using holistic model
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = holistic_model.process(image)
    image.flags.writeable = True
 
    # Converting back the RGB image to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
 
    
    mp_drawing.draw_landmarks(
        image, 
        results.pose_landmarks, 
        mp_holistic.POSE_CONNECTIONS,            
    )
    
    try:
        pose = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())

        pose_row.insert(0, class_name)
            
        with open(data_path, mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',')
            csv_writer.writerow(pose_row)
            
    except:
        cv2.putText(image, str('Error')
                    , (210,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    

    # Display the resulting image
    cv2.imshow("Test Data", image)
 
    # Enter key 'q' to break the loop
    if cv2.waitKey(5) & 0xFF == ord('q') :
        break
 
# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()

# TRAIN MODEL

In [107]:
model_name = "lag_raises_model"

In [108]:
# iterate over files in data directory
list_class = []
list_data = []
directory = modelFolerPath+"/"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        list_data.append(pd.read_csv(f))
        x = filename.split(".")
        list_class.append(x[0])

In [109]:
sys.stdout.write("[  There is " + str(len(list_class)) + " class for this model  ]\n")
print("==============================================================================")
for i in list_class:
    print(i)

print("==============================================================================")
df_model = pd.concat(list_data,ignore_index=True)
print("df_model is now complete")

[  There is 6 class for this model  ]
lag_raises_1
lag_raises_2
lag_raises_3
sit
stand
walk
df_model is now complete


In [110]:
df_model = df_model.drop(0)

In [111]:
df_model 

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
1,lag_raises_1,0.491479,0.151624,-0.286308,0.999143,0.502623,0.136857,-0.252909,0.998622,0.509427,...,0.263025,0.425274,0.515229,1.068508,-0.006923,0.774775,0.447526,1.033626,0.051697,0.752274
2,lag_raises_1,0.493231,0.150147,-0.327869,0.999196,0.503508,0.134814,-0.293247,0.998702,0.509248,...,0.294830,0.421253,0.516434,1.055524,0.032066,0.777759,0.457961,1.049469,0.086106,0.752219
3,lag_raises_1,0.495401,0.148811,-0.337888,0.999232,0.504919,0.133481,-0.304622,0.998761,0.510330,...,0.303240,0.425067,0.506203,1.046283,0.111449,0.772487,0.450746,1.019920,0.099775,0.750123
4,lag_raises_1,0.496997,0.152307,-0.327995,0.999207,0.505578,0.136507,-0.297223,0.998708,0.510929,...,0.283814,0.430700,0.513872,1.045367,0.121666,0.776842,0.460448,1.048157,0.090392,0.754101
5,lag_raises_1,0.492804,0.153481,-0.313507,0.999202,0.502115,0.137699,-0.283643,0.998699,0.507627,...,0.275206,0.430895,0.512038,1.054690,0.109831,0.770112,0.463354,1.060496,0.070919,0.749828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11938,walk,0.375721,0.074755,-0.203680,0.999436,0.369221,0.058373,-0.187952,0.998986,0.369507,...,0.149129,0.623300,0.461684,0.975862,0.336845,0.726455,0.300782,1.066792,0.009269,0.688695
11939,walk,0.405675,0.075380,-0.274981,0.999478,0.399794,0.057924,-0.257307,0.999063,0.400405,...,0.189618,0.644691,0.459132,0.983548,0.437276,0.744057,0.315667,1.067534,0.040548,0.714595
11940,walk,0.431521,0.075075,-0.294875,0.999518,0.426292,0.057489,-0.275408,0.999129,0.426570,...,0.205530,0.664382,0.453021,0.981769,0.446030,0.755130,0.377171,1.050129,0.056272,0.735070
11941,walk,0.442688,0.077119,-0.300460,0.999499,0.438099,0.059013,-0.280598,0.999048,0.438478,...,0.218055,0.683668,0.451181,0.980700,0.494980,0.751925,0.431793,1.000979,0.062790,0.751414


In [112]:
x = df_model .drop('class', axis=1) 
y = df_model ['class']

In [113]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=999)

In [114]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression(), verbose = 1),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier(), verbose = 1),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier(), verbose = 1),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier(), verbose = 1),
}

In [115]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s


C:\Users\Leopard\anaconda3\envs\AI4H\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[Pipeline]  (step 2 of 2) Processing logisticregression, total=   0.5s
[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline] ... (step 2 of 2) Processing ridgeclassifier, total=   0.0s
[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline]  (step 2 of 2) Processing randomforestclassifier, total=   4.4s
[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline]  (step 2 of 2) Processing gradientboostingclassifier, total= 2.9min


In [116]:
# list(pipelines.values())[3]
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())],
          verbose=1),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())],
          verbose=1),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())],
          verbose=1),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())],
          verbose=1)}

In [117]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.996232733361239
rc 0.9723733779824194
rf 0.9979070740895772
gb 0.9979070740895772


In [118]:
model_dst=prj_path+"\Model/"
save_dst=model_dst+'lag_raises.pkl'
pickle.dump(fit_models['rf'],open(save_dst, 'wb'))